In [64]:
import polars as pl
import pandas as pd
from src.procedures import fetch_last_data
from dotenv import dotenv_values
import datetime
from sqlalchemy import create_engine

import geopandas as gpd
from shapely.geometry import Point

config = dotenv_values("../.env")

In [65]:
hist_2023 = pd.read_csv("../data/viirs-yearly-summary/viirs-snpp_2023.csv")

file_path = "../data/IndonesianCitiesDistrictsUpdated.json"
adm_df = gpd.read_file(file_path)

display(hist_2023.head(3), adm_df.head(3))

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,0.45100,109.06416,334.46,0.42,0.38,1/1/2023,559,N,VIIRS,n,2.0NRT,288.34,27.04,D
1,-1.32986,113.40453,342.43,0.41,0.37,1/1/2023,559,N,VIIRS,n,2.0NRT,279.44,7.98,D
2,-0.84505,116.93285,341.42,0.41,0.45,1/1/2023,559,N,VIIRS,n,2.0NRT,286.10,9.12,D


,id,provinsi,geometry
0,Aceh Barat,Aceh,"POLYGON ((96.49109 4.62277, 96.49329 4.56110, ..."
1,Aceh Barat Daya,Aceh,"POLYGON ((97.15358 3.74559, 97.14205 3.73411, ..."
2,Aceh Besar,Aceh,"MULTIPOLYGON (((95.17371 5.55513, 95.15416 5.5..."


In [66]:
# change datetype format
hist_2023["acq_date"] = pd.to_datetime(hist_2023["acq_date"]).dt.strftime('%Y-%m-%d')

# replave values
hist_2023["confidence"] = hist_2023["confidence"].replace({
    "n":"Nominal", "h":"High", "l":"Low"
})

hist_2023["daynight"] = hist_2023["daynight"].replace({
    "D":"Day", "N":"Night"
})


hist_2023.head(3)

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,0.45100,109.06416,334.46,0.42,0.38,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,288.34,27.04,Day
1,-1.32986,113.40453,342.43,0.41,0.37,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,279.44,7.98,Day
2,-0.84505,116.93285,341.42,0.41,0.45,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,286.10,9.12,Day


In [67]:
# Zip lat-lon as tuple, convert to Points object
hist_2023["coords"] = list(zip(hist_2023["longitude"], hist_2023["latitude"]))
hist_2023["coords"] = hist_2023["coords"].apply(Point)

# Turn into geodataframe, perform spatial join
points = gpd.GeoDataFrame(hist_2023, geometry="coords")
joined_df = gpd.tools.sjoin(points, adm_df, predicate="within", how='left')

joined_df.head(3)

C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_18584\2749868562.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_df = gpd.tools.sjoin(points, adm_df, predicate="within", how='left')


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,coords,index_right,id,provinsi
0,0.45100,109.06416,334.46,0.42,0.38,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,288.34,27.04,Day,POINT (109.06416 0.45100),337.0,Mempawah,Kalimantan Barat
1,-1.32986,113.40453,342.43,0.41,0.37,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,279.44,7.98,Day,POINT (113.40453 -1.32986),111.0,Gunung Mas,Kalimantan Tengah
2,-0.84505,116.93285,341.42,0.41,0.45,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,286.10,9.12,Day,POINT (116.93285 -0.84505),279.0,Kutai Kartanegara,Kalimantan Timur


In [68]:
# Rename some columns, drop the unnecessary ones for the analysis
joined_df = joined_df.rename(columns={
    "id":"second_adm", "provinsi":"first_adm"
})

joined_df = joined_df.drop(["scan", "track", "bright_t31", "coords", "index_right"], axis=1)

joined_df.head()

,latitude,longitude,brightness,acq_date,acq_time,satellite,instrument,confidence,version,frp,daynight,second_adm,first_adm
0,0.45100,109.06416,334.46,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,27.04,Day,Mempawah,Kalimantan Barat
1,-1.32986,113.40453,342.43,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,7.98,Day,Gunung Mas,Kalimantan Tengah
2,-0.84505,116.93285,341.42,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,9.12,Day,Kutai Kartanegara,Kalimantan Timur
3,-1.47823,113.02875,329.07,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,2.06,Day,Katingan,Kalimantan Tengah
4,-1.43891,114.12376,327.32,2023-01-01,559,N,VIIRS,Nominal,2.0NRT,2.43,Day,Kapuas,Kalimantan Tengah


In [71]:
pl_joined_df = pl.from_pandas(joined_df)
pl_joined_df = pl_joined_df.with_columns(
    pl.col("acq_date").str.strptime(pl.Date, "%Y-%m-%d")
)

pl_joined_df.head(3)

latitude,longitude,brightness,acq_date,acq_time,satellite,instrument,confidence,version,frp,daynight,second_adm,first_adm
f64,f64,f64,date,i64,str,str,str,str,f64,str,str,str
0.451,109.06416,334.46,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",27.04,"""Day""","""Mempawah""","""Kalimantan Bar…"
-1.32986,113.40453,342.43,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",7.98,"""Day""","""Gunung Mas""","""Kalimantan Ten…"
-0.84505,116.93285,341.42,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",9.12,"""Day""","""Kutai Kartaneg…","""Kalimantan Tim…"


In [72]:
connection = config.get("CONNECTION_URI")

# write to database
pl_joined_df.write_database(table_name="processed_viirs",  connection=connection, if_exists="append")

In [73]:
pl_joined_df

latitude,longitude,brightness,acq_date,acq_time,satellite,instrument,confidence,version,frp,daynight,second_adm,first_adm
f64,f64,f64,date,i64,str,str,str,str,f64,str,str,str
0.451,109.06416,334.46,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",27.04,"""Day""","""Mempawah""","""Kalimantan Bar…"
-1.32986,113.40453,342.43,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",7.98,"""Day""","""Gunung Mas""","""Kalimantan Ten…"
-0.84505,116.93285,341.42,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",9.12,"""Day""","""Kutai Kartaneg…","""Kalimantan Tim…"
-1.47823,113.02875,329.07,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",2.06,"""Day""","""Katingan""","""Kalimantan Ten…"
-1.43891,114.12376,327.32,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",2.43,"""Day""","""Kapuas""","""Kalimantan Ten…"
-1.44242,114.12428,333.4,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",2.43,"""Day""","""Kapuas""","""Kalimantan Ten…"
-1.44157,114.38811,332.27,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",2.43,"""Day""","""Kapuas""","""Kalimantan Ten…"
-1.55646,113.68083,327.89,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",3.69,"""Day""","""Gunung Mas""","""Kalimantan Ten…"
-1.70522,113.16237,333.01,2023-01-01,559,"""N""","""VIIRS""","""Nominal""","""2.0NRT""",1.85,"""Day""","""Katingan""","""Kalimantan Ten…"
